In [1]:
print("Loading...")

# common libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import os
from os import listdir
from os.path import isfile, join

# CV and Image
import cv2
from PIL import Image

# pickle
import pickle

# keras
import keras
from keras import optimizers
from keras import backend as K
from keras.models import Model, Sequential, load_model
from keras.layers import *
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
K.image_data_format()

print("Done")

Loading...
Done


In [ ]:
p = "dataset/"

print("These are the folders we'll be working with :")

for f in os.listdir(p):
    print(f)

In [19]:
def load_images(folder):
    
    train_data=[]
    
    for filename in os.listdir(folder):
        
        if filename != ".directory" :

            image = cv2.imread(os.path.join(folder,filename),cv2.IMREAD_GRAYSCALE)
            image = ~image

            if image is not None:

                ret, thresh = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
                contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
                contour = sorted(contours, key = lambda ctr: cv2.boundingRect(ctr)[0])

                a = int(28)
                b = int(28)
                maxi = 0

                for c in contour:

                    x,y,a,b=cv2.boundingRect(c)
                    maxi=max(a*b,maxi)

                    if maxi==a*b:

                        x_max=x
                        y_max=y
                        w_max=a
                        h_max=b

                im_crop = thresh[y_max:y_max+h_max+10, x_max:x_max+w_max+10]
                im_resize = cv2.resize(im_crop,(28,28))
                im_resize = np.reshape(im_resize,(784,1))
                train_data.append(im_resize)
            
    return train_data

In [ ]:
data = []

data = load_images(p+"0")
for i in range(0, len(data)):
    data[i] = np.append(data[i], ['0'])
print(len(data))

In [ ]:
iter = 0

for i in list(os.listdir(p)) :
    
    if i not in ["0", ".directory"] :
    
        print("Iter :", iter)
        print("Working with the", i, "folder")

        data_i = load_images(p+i)

        if i in [str(k) for k in range(1, 10)] :

            for j in range(0, len(data_i)):
                data_i[j] = np.append(data_i[j], [i])

        if i == "add" :

            for j in range(0, len(data_i)):
                data_i[j] = np.append(data_i[j], ["10"])

        if i == "sub" :

            for j in range(0, len(data_i)):
                data_i[j] = np.append(data_i[j], ["11"])


        if i == "mul" :

            for j in range(0, len(data_i)):
                data_i[j] = np.append(data_i[j], ["12"])

        if i == "div" :

            for j in range(0, len(data_i)):
                data_i[j] = np.append(data_i[j], ["13"])

        if i == "eq" :

            for j in range(0, len(data_i)):
                data_i[j] = np.append(data_i[j], ["14"])                

        if i == "dec" :

            for j in range(0, len(data_i)):
                data_i[j] = np.append(data_i[j], ["15"])

        if i == "x" :

            for j in range(0, len(data_i)):
                data_i[j] = np.append(data_i[j], ["16"])

        if i == "y" :

            for j in range(0, len(data_i)):
                data_i[j] = np.append(data_i[j], ["17"]) 


        if i == "z" :

            for j in range(0, len(data_i)):
                data_i[j] = np.append(data_i[j], ["18"]) 


        data = np.concatenate((data, data_i))
        print(len(data))

        print()

        iter += 1

In [ ]:
df=pd.DataFrame(data,index=None)
df.to_csv('train_handwritten.csv',index=False)

data = pd.read_csv('train_handwritten.csv',index_col=False)
labels = data[['784']]

data.drop(data.columns[[784]],axis=1,inplace=True)
data.head()

In [ ]:
np.random.seed(1212)
labels=np.array(labels)
cat=to_categorical(labels,num_classes=19)
cat[0]

In [ ]:
temp=data.to_numpy()
X_train = temp.reshape(temp.shape[0], 28, 28, 1)
X_train.shape

In [25]:
l=[]
for i in range(X_train.shape[0]):
    l.append(np.array(data[i:i+1]).reshape(1,28,28))

np.random.seed(7)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization

model = Sequential()

# Primo strato convoluzionale
model.add(Conv2D(64, (3, 3), input_shape=(28, 28, 1), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Secondo strato convoluzionale
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Terzo strato convoluzionale
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Flattening
model.add(Flatten())

# Strati densi
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(19, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_layer_names=True)

In [ ]:
checkpoint = ModelCheckpoint("eq_solver.keras", monitor='accuracy', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
nn = model.fit(X_train, cat, epochs=500, batch_size=256, shuffle=True, verbose=1, callbacks=[checkpoint]).history

In [ ]:
plt.plot(nn['loss'])
plt.plot(nn['accuracy'])
plt.title('model loss')
plt.ylabel('loss and accuracy')
plt.xlabel('epoch')
plt.legend(['train_loss', 'train_accuracy'], loc='upper right')